In [1]:
### USFWS_Goose_Mark-Resight_DataExtract.py
### Version: 6/2/2022
### Author: Khem So, khem_so@fws.gov, (503) 231-6839
### Abstract: This Python 3 script pulls data from the USFWS_Goose_Mark-Resight ArcGIS Online feature service and performs joins and merges to result in combined datasets matching the Migratory Birds program template for dusky Canada goose mark-resight data and refuge/location-specific extracts.

In [2]:
import arcpy
import pandas as pd
from arcgis import GIS
import time, os, fnmatch, shutil
import janitor # https://pyjanitor-devs.github.io/pyjanitor/
import openpyxl # https://openpyxl.readthedocs.io/

In [3]:
### ArcGIS Online stores date-time information in UTC by default. This function uses the pytz package to convert time zones and can be used to convert from UTC ("UTC") to localized time. For example, localized "US/Pacific" is either Pacific Standard Time UTC-8 or Pacific Daylight Time UTC-7 depending upon time of year.
from datetime import datetime
from pytz import timezone
def change_timezone_of_field(df, source_date_time_field, new_date_time_field_suffix, source_timezone, new_timezone):
    """Returns the values in *source_date_time_field* with its timezone converted to a new timezone within a new field *new_date_time_field*
    : param df: The name of the spatially enabled or pandas DataFrame containing datetime fields
    : param source_date_time_field: The name of the datetime field whose timezone is to be changed
    : param new_date_time_field_suffix: Suffix appended to the end of the name of the source datetime field. This is used to create the new date time field name.
    : param source_timezone: The name of the source timezone
    : param new_timezone: The name of the converted timezone. For possible values, see https://gist.github.com/heyalexej/8bf688fd67d7199be4a1682b3eec7568
    """
    # Define the source timezone in the source_date_time_field
    df[source_date_time_field] = df[source_date_time_field].dt.tz_localize(source_timezone)
    # Define the name of the new date time field
    new_date_time_field = source_date_time_field + new_date_time_field_suffix
    # Convert the datetime in the source_date_time_field to the new timezone in a new field called new_date_time_field
    df[new_date_time_field] = df[source_date_time_field].dt.tz_convert(new_timezone)

In [4]:
### This function converts Python datetime64 fields to %m/%d/%Y %H:%M:%S %Z%z format
def archive_dt_field(df):
    """Selects fields with data types of 'datetime64[ns, UTC]','datetime64[ns, US/Pacific]' and converts to %m/%d/%Y %H:%M:%S %Z%z format for archiving to Excel
    : param df: The name of the spatially enabled or pandas DataFrame containing datetime fields
    """
    archive_dt_field_list = df.select_dtypes(include=['datetime64[ns, UTC]','datetime64[ns, US/Pacific]'])
    for col in archive_dt_field_list:
        df[col] = df[col].dt.strftime('%m/%d/%Y %H:%M:%S %Z%z')

In [5]:
### This function converts strings in a dataframe to uppercase
def upper_consistent(df):
    df = df.apply(lambda x: x.str.upper() if x.dtype == "object" else x) 
    return df

In [6]:
### Allow authentication via login to U.S. Fish & Wildlife Service ArcGIS Online account via ArcGIS Pro
gis = GIS("pro")

In [7]:
### Enter start and end dates of interest
# uncomment next line to use ArcGIS interface, otherwise hard coding year
# startdate = arcpy.GetParameterAsText(0)
# enddate = arcpy.GetParameterAsText(1)
start_date = "09-01-2021"
end_date = "06-01-2022"

In [8]:
### Enter path for local file saving
# uncomment next line to use ArcGIS interface, otherwise hard coding out_workspace
# out_workspace = arcpy.GetParameterAsText(1)
out_workspace = "C:/Users/kso/Desktop/"

In [9]:
### Create timestamp for file naming
t = time.localtime()
timestamp = time.strftime('%Y-%m-%d_%H%M', t)

In [10]:
### Paths to ArcGIS Online data
# To populate Service ItemId, go to Feature Service webpage and in bottom right corner, click on the View link.
# Current Feature Service webpage: https://fws.maps.arcgis.com/home/item.html?id=87e3dfd8e8974fac84d29b7092025a0d
ServiceItemID = gis.content.get("87e3dfd8e8974fac84d29b7092025a0d")

### There are separate methods for pulling spatial versus non-spatial data into Python. Spatial layers will become Spatially Enabled DataFrame objects. Non-spatial data will become regular pandas DataFrame objects.
## Define variables pointing to spatial layers
MetadataLyr = ServiceItemID.layers[0]
ObservationPointLyr = ServiceItemID.layers[1]
## Create Spatially Enabled DataFrame objects
sedfMetadata = pd.DataFrame.spatial.from_layer(MetadataLyr)
sedfObservationPoint = pd.DataFrame.spatial.from_layer(ObservationPointLyr)

## Define variables point to non-spatial (tabular) data
OtherSpeciesBands = r"https://services.arcgis.com/QVENGdaPbd4LUkLV/ArcGIS/rest/services/service_e01fb68477c047f4ab25ad3e6c30ac1b/FeatureServer/2"

## Convert AGOL table to NumPy Array and then to pandas DataFrames
naOtherSpeciesBands = arcpy.da.TableToNumPyArray(OtherSpeciesBands,["objectid","globalid","SpeciesText","BandNote","OtherSpBandCode","OtherBandJoin","parentglobalid","CreationDate","Creator","EditDate","Editor"])
dfOtherSpeciesBands = pd.DataFrame(naOtherSpeciesBands)

In [11]:
### Filter sedfMetadata by date range
sedfMetadataYYYY = sedfMetadata.filter_date("EffortDate", start_date, end_date)

In [12]:
### Use change_timezone_of_field function to convert all datetime fields in dataframe from UTC to Pacific within new field with _Pacific suffix
for col in sedfMetadata.columns:
     if sedfMetadata[col].dtype == 'datetime64[ns]':
         change_timezone_of_field(sedfMetadata, col, "_Pacific", "UTC", "US/Pacific")

for col in sedfObservationPoint.columns:
     if sedfObservationPoint[col].dtype == 'datetime64[ns]':
         change_timezone_of_field(sedfObservationPoint, col, "_Pacific", "UTC", "US/Pacific")

for col in dfOtherSpeciesBands.columns:
     if dfOtherSpeciesBands[col].dtype == 'datetime64[ns]':
         change_timezone_of_field(dfOtherSpeciesBands, col, "_Pacific", "UTC", "US/Pacific")

In [13]:
### Export raw data frames as backup
## Use archive_dt_field function to convert Python date time into format Excel can read more easily
archive_dt_field(sedfMetadata)
archive_dt_field(sedfObservationPoint)
archive_dt_field(dfOtherSpeciesBands)

## Create export paths for backup and writes to Excel spreadsheet
writer = pd.ExcelWriter(os.path.join(out_workspace,('USFWS_Goose_Mark-Resight_BKUP_' + timestamp + '.xlsx')))
sedfMetadata.to_excel(writer, 'Metadata')
sedfObservationPoint.to_excel(writer, 'ObservationPoint')
dfOtherSpeciesBands.to_excel(writer, 'OtherSpeciesBands')
writer.save()

In [14]:
### Populate ObserverText column
sedfMetadataYYYY["ObserverText"] = sedfMetadataYYYY["Observer"]
sedfMetadataYYYY.loc[sedfMetadataYYYY["Observer"] == "Other", "ObserverText"] = sedfMetadataYYYY["ObserverOther"]

In [15]:
### Populate State column based on Refuge
WA_filter = ['Willapa NWR', 'Julia Butler Hansen Refuge for the Columbian White-tailed Deer', 'Ridgefield NWR']
OR_filter = ['Lewis and Clark NWR', 'Tualatin River NWR', 'Wapato Lake NWR', 'Baskett Slough NWR', 'Ankeny NWR', 'William L. Finley NWR', 'Nestucca Bay NWR']
sedfMetadataYYYY.loc[sedfMetadataYYYY["SiteName"].isin(WA_filter), "State"] = "WA"
sedfMetadataYYYY.loc[sedfMetadataYYYY["SiteName"].isin(OR_filter), "State"] = "OR"

In [16]:
### Join sedfMetadataYYYY with sedfObservationPoint
sedfMetadataYYYY_ObservationPoint = pd.merge(sedfMetadataYYYY,sedfObservationPoint, how="inner", left_on="globalid", right_on="parentglobalid")
sedfMetadataYYYY_ObservationPoint

,objectid_x,globalid_x,SiteName,SiteNameOther,Observer,ObserverOther,ObserverText,State,EffortDate,EffortDate_Text,...,UnknownGooseCount,UnknownSwanCount,WesternCount,WhiteFrontedCount,WuskyCount,globalid_y,objectid_y,parentglobalid,CreationDate_Pacific,EditDate_Pacific
0,2,c4b960e1-5770-45da-b722-c7b240377069,Willapa NWR,None,William Ritchie,None,William Ritchie,WA,2021-10-04 19:00:00,10/04/2021,...,0.0,0,0.0,0,0.0,75497ba2-41a0-4e3d-9da8-55404c50970b,2,c4b960e1-5770-45da-b722-c7b240377069,10/04/2021 15:18:42 PDT-0700,10/04/2021 15:18:42 PDT-0700
1,2,c4b960e1-5770-45da-b722-c7b240377069,Willapa NWR,None,William Ritchie,None,William Ritchie,WA,2021-10-04 19:00:00,10/04/2021,...,0.0,0,0.0,0,0.0,7cc6aae9-3d1e-49d8-a617-e0fc1dfe8226,3,c4b960e1-5770-45da-b722-c7b240377069,10/04/2021 15:18:42 PDT-0700,10/04/2021 15:18:42 PDT-0700
2,2,c4b960e1-5770-45da-b722-c7b240377069,Willapa NWR,None,William Ritchie,None,William Ritchie,WA,2021-10-04 19:00:00,10/04/2021,...,0.0,0,0.0,0,0.0,505c5601-051d-496a-a141-0179d1535d1c,4,c4b960e1-5770-45da-b722-c7b240377069,10/04/2021 15:18:42 PDT-0700,10/04/2021 15:18:42 PDT-0700
3,2,c4b960e1-5770-45da-b722-c7b240377069,Willapa NWR,None,William Ritchie,None,William Ritchie,WA,2021-10-04 19:00:00,10/04/2021,...,0.0,0,0.0,0,0.0,ea7a4817-7cf7-419c-b11a-87a7644a117c,5,c4b960e1-5770-45da-b722-c7b240377069,10/04/2021 15:18:42 PDT-0700,10/04/2021 15:18:42 PDT-0700
4,2,c4b960e1-5770-45da-b722-c7b240377069,Willapa NWR,None,William Ritchie,None,William Ritchie,WA,2021-10-04 19:00:00,10/04/2021,...,0.0,0,0.0,0,0.0,a6d6f117-6782-4a35-8f60-5cc13e006082,6,c4b960e1-5770-45da-b722-c7b240377069,10/04/2021 15:18:42 PDT-0700,10/04/2021 15:18:42 PDT-0700
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1037,90,75c1c529-a4f8-4cbb-afda-cfd50d4ed76f,Julia Butler Hansen Refuge for the Columbian W...,None,Stephanie Honcharenko,None,Stephanie Honcharenko,WA,2022-04-18 19:00:00,4/18/2022,...,0.0,0,2.0,0,0.0,8f2f1159-32eb-4b58-8cd2-b71e80fc116b,1042,75c1c529-a4f8-4cbb-afda-cfd50d4ed76f,04/26/2022 11:29:59 PDT-0700,04/26/2022 11:29:59 PDT-0700
1038,90,75c1c529-a4f8-4cbb-afda-cfd50d4ed76f,Julia Butler Hansen Refuge for the Columbian W...,None,Stephanie Honcharenko,None,Stephanie Honcharenko,WA,2022-04-18 19:00:00,4/18/2022,...,0.0,0,3.0,0,0.0,3afaa7c2-05fd-4e15-ba7e-12de574d0a17,1043,75c1c529-a4f8-4cbb-afda-cfd50d4ed76f,04/26/2022 11:29:59 PDT-0700,04/26/2022 11:29:59 PDT-0700
1039,90,75c1c529-a4f8-4cbb-afda-cfd50d4ed76f,Julia Butler Hansen Refuge for the Columbian W...,None,Stephanie Honcharenko,None,Stephanie Honcharenko,WA,2022-04-18 19:00:00,4/18/2022,...,0.0,0,7.0,0,0.0,106c3bd9-a471-405f-b46b-1b8fc8ec219a,1044,75c1c529-a4f8-4cbb-afda-cfd50d4ed76f,04/26/2022 11:29:59 PDT-0700,04/26/2022 11:29:59 PDT-0700
1040,90,75c1c529-a4f8-4cbb-afda-cfd50d4ed76f,Julia Butler Hansen Refuge for the Columbian W...,None,Stephanie Honcharenko,None,Stephanie Honcharenko,WA,2022-04-18 19:00:00,4/18/2022,...,0.0,0,1.0,0,0.0,a64476e0-38ea-4291-b3b3-488b511b64cc,1045,75c1c529-a4f8-4cbb-afda-cfd50d4ed76f,04/26/2022 11:29:59 PDT-0700,04/26/2022 11:29:59 PDT-0700


In [17]:
### Populate LatitudeDD and LongitudeDD fields based on whether there are prepopulated values
Prepopulated_filter = ['Ridgefield NWR', 'Tualatin River NWR', 'Wapato Lake NWR']
sedfMetadataYYYY_ObservationPoint.loc[sedfMetadataYYYY_ObservationPoint["SiteName"].isin(Prepopulated_filter) & sedfMetadataYYYY_ObservationPoint["Latitude_Prepopulated"].notnull(), "LatitudeDD"] = sedfMetadataYYYY_ObservationPoint["Latitude_Prepopulated"]
sedfMetadataYYYY_ObservationPoint.loc[sedfMetadataYYYY_ObservationPoint["SiteName"].isin(Prepopulated_filter) & sedfMetadataYYYY_ObservationPoint["Longitude_Prepopulated"].notnull(), "LongitudeDD"] = sedfMetadataYYYY_ObservationPoint["Longitude_Prepopulated"]
sedfMetadataYYYY_ObservationPoint.loc[sedfMetadataYYYY_ObservationPoint["SiteName"].isin(Prepopulated_filter) & sedfMetadataYYYY_ObservationPoint["Latitude_Prepopulated"].isnull(), "LatitudeDD"] = sedfMetadataYYYY_ObservationPoint["Latitude"]
sedfMetadataYYYY_ObservationPoint.loc[sedfMetadataYYYY_ObservationPoint["SiteName"].isin(Prepopulated_filter) & sedfMetadataYYYY_ObservationPoint["Longitude_Prepopulated"].isnull(), "LongitudeDD"] = sedfMetadataYYYY_ObservationPoint["Longitude"]
sedfMetadataYYYY_ObservationPoint.loc[~sedfMetadataYYYY_ObservationPoint["SiteName"].isin(Prepopulated_filter), "LatitudeDD"] = sedfMetadataYYYY_ObservationPoint["Latitude"]
sedfMetadataYYYY_ObservationPoint.loc[~sedfMetadataYYYY_ObservationPoint["SiteName"].isin(Prepopulated_filter), "LongitudeDD"] = sedfMetadataYYYY_ObservationPoint["Longitude"]

In [18]:
### Calculate column for sum total of count data
col_list= ['DuskyCount', 'WuskyCount', 'WesternCount', 'TavLessCount', 'TavCount', 'LessCount', 'AleutianCount', 'CacklingCount', 'UnknownCanadaCount', 'UnknownCacklerCount', 'WhiteFrontedCount', 'SnowCount', 'RossCount', 'UnknownGooseCount', 'TrumpeterCount', 'TundraCount', 'UnknownSwanCount']
sedfMetadataYYYY_ObservationPoint["Total"] = sedfMetadataYYYY_ObservationPoint[col_list].sum(axis=1)

In [19]:
### Create data frame for dusky neckband/collar data
dfDuskyNeckband = sedfMetadataYYYY_ObservationPoint.copy()
### Reorder columns
dfDuskyNeckband = dfDuskyNeckband[['objectid_x', 'globalid_x', 'SiteName', 'SiteNameOther', 'Observer', 'ObserverOther', 'ObserverText', 'State', 'EffortDate', 'EffortDate_Text', 'FormName', 'survey_uuid', 'FormVersion', 'CreationDate_x', 'Creator_x', 'EditDate_x', 'Editor_x', 'SHAPE_x', 'AleutianCount', 'CacklingCount', 'CreationDate_y', 'Creator_y', 'DuskyBandNote', 'DuskyCollar1', 'DuskyCollar2', 'DuskyCollar3', 'DuskyCollar4', 'DuskyCollar5', 'DuskyCollar6', 'DuskyCollar7', 'DuskyCollar8', 'DuskyCollar9', 'DuskyCollar10', 'DuskyCollar11', 'DuskyCollar12', 'DuskyCollar13', 'DuskyCollar14', 'DuskyCollar15', 'DuskyCollar16', 'DuskyCollar17', 'DuskyCollar18', 'DuskyCollar19', 'DuskyCollar20', 'DuskyCollar21', 'DuskyCollar22', 'DuskyCollar23', 'DuskyCollar24', 'DuskyCollar25', 'DuskyCollar26', 'DuskyCollar27', 'DuskyCollar28', 'DuskyCollar29', 'DuskyCollar30', 'DuskyCount', 'EditDate_y', 'Editor_y', 'EffortNotes', 'EffortTime', 'Latitude', 'Latitude_Prepopulated', 'LessCount', 'LocationDescription', 'LocationDescription_ReadOnly', 'LocationOther', 'Longitude', 'Longitude_Prepopulated', 'NumberDuskyCollars', 'RossCount', 'SHAPE_y', 'SnowCount', 'TavCount', 'TavLessCount', 'TotalGeese', 'TrumpeterCount', 'TundraCount', 'UnknownCacklerCount', 'UnknownCanadaCount', 'UnknownGooseCount', 'UnknownSwanCount', 'WesternCount', 'WhiteFrontedCount', 'WuskyCount', 'globalid_y', 'objectid_y', 'parentglobalid', 'CreationDate_Pacific', 'EditDate_Pacific', 'LatitudeDD', 'LongitudeDD', 'Total']]
### Drop columns with null values
dfDuskyNeckband.dropna(how='all', axis=1, inplace=True)
### Calculate field for coordinate precision
dfDuskyNeckband["Prec"] = '0'
#dfDuskyNeckband["Coord1"] = dfDuskyNeckband["LatitudeDD"].replace(".", "?") 
#dfDuskyNeckband["Format"] = "DD"
dfDuskyNeckband["Location"] = dfDuskyNeckband.SiteName.str.cat(dfDuskyNeckband.LocationDescription,sep=" ", na_rep = "")
dfDuskyNeckband

,objectid_x,globalid_x,SiteName,Observer,ObserverOther,ObserverText,State,EffortDate,EffortDate_Text,FormName,...,globalid_y,objectid_y,parentglobalid,CreationDate_Pacific,EditDate_Pacific,LatitudeDD,LongitudeDD,Total,Prec,Location
0,2,c4b960e1-5770-45da-b722-c7b240377069,Willapa NWR,William Ritchie,None,William Ritchie,WA,2021-10-04 19:00:00,10/04/2021,Dusky Goose Survey,...,75497ba2-41a0-4e3d-9da8-55404c50970b,2,c4b960e1-5770-45da-b722-c7b240377069,10/04/2021 15:18:42 PDT-0700,10/04/2021 15:18:42 PDT-0700,46.365221,-123.994647,0.0,0,Willapa NWR Riekkola Salt Marsh SE
1,2,c4b960e1-5770-45da-b722-c7b240377069,Willapa NWR,William Ritchie,None,William Ritchie,WA,2021-10-04 19:00:00,10/04/2021,Dusky Goose Survey,...,7cc6aae9-3d1e-49d8-a617-e0fc1dfe8226,3,c4b960e1-5770-45da-b722-c7b240377069,10/04/2021 15:18:42 PDT-0700,10/04/2021 15:18:42 PDT-0700,46.370991,-123.995393,0.0,0,Willapa NWR Riekkola Salt Marsh NE
2,2,c4b960e1-5770-45da-b722-c7b240377069,Willapa NWR,William Ritchie,None,William Ritchie,WA,2021-10-04 19:00:00,10/04/2021,Dusky Goose Survey,...,505c5601-051d-496a-a141-0179d1535d1c,4,c4b960e1-5770-45da-b722-c7b240377069,10/04/2021 15:18:42 PDT-0700,10/04/2021 15:18:42 PDT-0700,46.365704,-124.006726,5.0,0,Willapa NWR Riekkola South Field #3 (Parker Sl.)
3,2,c4b960e1-5770-45da-b722-c7b240377069,Willapa NWR,William Ritchie,None,William Ritchie,WA,2021-10-04 19:00:00,10/04/2021,Dusky Goose Survey,...,ea7a4817-7cf7-419c-b11a-87a7644a117c,5,c4b960e1-5770-45da-b722-c7b240377069,10/04/2021 15:18:42 PDT-0700,10/04/2021 15:18:42 PDT-0700,46.362190,-123.995814,0.0,0,Willapa NWR Riekkola South Field #1 (67th Place)
4,2,c4b960e1-5770-45da-b722-c7b240377069,Willapa NWR,William Ritchie,None,William Ritchie,WA,2021-10-04 19:00:00,10/04/2021,Dusky Goose Survey,...,a6d6f117-6782-4a35-8f60-5cc13e006082,6,c4b960e1-5770-45da-b722-c7b240377069,10/04/2021 15:18:42 PDT-0700,10/04/2021 15:18:42 PDT-0700,46.403080,-123.951040,0.0,0,Willapa NWR Omeara Point (incl. Quarry Marsh)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1037,90,75c1c529-a4f8-4cbb-afda-cfd50d4ed76f,Julia Butler Hansen Refuge for the Columbian W...,Stephanie Honcharenko,None,Stephanie Honcharenko,WA,2022-04-18 19:00:00,4/18/2022,Dusky Goose Survey,...,8f2f1159-32eb-4b58-8cd2-b71e80fc116b,1042,75c1c529-a4f8-4cbb-afda-cfd50d4ed76f,04/26/2022 11:29:59 PDT-0700,04/26/2022 11:29:59 PDT-0700,46.232409,-123.402388,2.0,0,Julia Butler Hansen Refuge for the Columbian W...
1038,90,75c1c529-a4f8-4cbb-afda-cfd50d4ed76f,Julia Butler Hansen Refuge for the Columbian W...,Stephanie Honcharenko,None,Stephanie Honcharenko,WA,2022-04-18 19:00:00,4/18/2022,Dusky Goose Survey,...,3afaa7c2-05fd-4e15-ba7e-12de574d0a17,1043,75c1c529-a4f8-4cbb-afda-cfd50d4ed76f,04/26/2022 11:29:59 PDT-0700,04/26/2022 11:29:59 PDT-0700,46.241310,-123.403597,3.0,0,Julia Butler Hansen Refuge for the Columbian W...
1039,90,75c1c529-a4f8-4cbb-afda-cfd50d4ed76f,Julia Butler Hansen Refuge for the Columbian W...,Stephanie Honcharenko,None,Stephanie Honcharenko,WA,2022-04-18 19:00:00,4/18/2022,Dusky Goose Survey,...,106c3bd9-a471-405f-b46b-1b8fc8ec219a,1044,75c1c529-a4f8-4cbb-afda-cfd50d4ed76f,04/26/2022 11:29:59 PDT-0700,04/26/2022 11:29:59 PDT-0700,46.253136,-123.433117,7.0,0,Julia Butler Hansen Refuge for the Columbian W...
1040,90,75c1c529-a4f8-4cbb-afda-cfd50d4ed76f,Julia Butler Hansen Refuge for the Columbian W...,Stephanie Honcharenko,None,Stephanie Honcharenko,WA,2022-04-18 19:00:00,4/18/2022,Dusky Goose Survey,...,a64476e0-38ea-4291-b3b3-488b511b64cc,1045,75c1c529-a4f8-4cbb-afda-cfd50d4ed76f,04/26/2022 11:29:59 PDT-0700,04/26/2022 11:29:59 PDT-0700,46.234221,-123.410579,1.0,0,Julia Butler Hansen Refuge for the Columbian W...


In [20]:
# Make sure that Dusky collars are uppercase
dfDuskyColumns = dfDuskyNeckband.filter(regex='Dusky*',axis=1)
dfDuskyColumnsList = list(dfDuskyColumns.columns.values)

dfDuskyNeckband[dfDuskyColumnsList] = upper_consistent(dfDuskyNeckband[dfDuskyColumnsList])

# Select only rows where DuskyCollar is not null
dfDuskyNeckband = dfDuskyNeckband[~dfDuskyNeckband.filter(like='DuskyCollar').isna().all(1)]

In [21]:
# Create data frame for export to Refuge staff. Use Darwin Core/biotic observation minimum standards.
dfDuskyNeckbandNWR = (
    pd.DataFrame(dfDuskyNeckband)
    .select_columns(['globalid_y', 'SiteName', 'LocationDescription', 'EffortDate_Text', 'EffortTime', 'ObserverText',  'Dusky*', 'Total', 'LatitudeDD', 'LongitudeDD'])
    .rename_columns(new_column_names={"globalid_y": "occurrenceID", "SiteName": "location", "EffortTime": "eventTime", "EffortDate_Text": "eventDate", "LatitudeDD": "decimalLatitude", "LongitudeDD": "decimalLongitude", "ObserverText": "recordedBy"})
)

In [22]:
# Create data frame for export to Migratory Birds. Use Migratory Birds schema.
dfDuskyNeckbandMB = (
    pd.DataFrame(dfDuskyNeckband)
    .select_columns(['globalid_y', 'State', 'EffortDate_Text', 'EffortTime', 'Prec', 'LatitudeDD', 'LongitudeDD', 'ObserverText', 'SiteName', 'LocationDescription', 'Location', 'Dusky*', 'Total'])
    .rename_columns(new_column_names={"globalid_y": "globalid", "EffortDate_Text": "Date", "LatitudeDD": "Latdd", "LongitudeDD": "Longdd", "ObserverText": "Obs", "DuskyCount": "Pres"})
)

In [23]:
### Pivot flock data from wide to long so that each row represents a separate species count, per occurrence
dfFlockPivotTemp = (
    pd.DataFrame(sedfObservationPoint)
    .select_columns(['globalid', 'DuskyCount', 'WuskyCount', 'WesternCount', 'TavLessCount', 'TavCount', 'LessCount', 'AleutianCount', 'CacklingCount', 'UnknownCanadaCount', 'UnknownCacklerCount', 'WhiteFrontedCount', 'SnowCount', 'RossCount', 'UnknownGooseCount', 'TrumpeterCount', 'TundraCount', 'UnknownSwanCount'])
    .pivot_longer(
    index = 'globalid',
    names_to = ('shortName', 'dimension'),
    names_sep = 'Count',
    sort_by_appearance = True,
    )
    .remove_columns(['dimension'])
    .filter_on("value != 0")
    .dropna()
)

In [24]:
### Use dictionaries and pandas map function to create new fields for scientific name, FWS taxon code, and ITIS code, based on the short species/subspecies name 
sciname_dict = {
    'Aleutian': 'Branta hutchinsii leucopareia',
    'Cackling': 'Branta hutchinsii minima',
    'Dusky': 'Branta canadensis occidentalis',
    'Less': 'Branta canadensis parvipes',
    'Tav': 'Branta hutchinsii taverneri',
    'TavLess': 'Branta hutchinsii taverneri x Branta canadensis parvipes',
    'Western': 'Branta canadensis moffitti',
    'Wusky': 'Branta canadensis moffitti x Branta canadensis occidentalis',
    'UnknownCackler': 'Branta hutchinsii',
    'UnknownCanada': 'Branta canadensis',
    'Ross': 'Chen rossii',
    'Snow': 'Chen caerulescens',
    'WhiteFronted': 'Anser albifrons',
    'UnknownGoose': 'Branta',
    'Trumpeter': 'Cygnus buccinator',
    'UnknownSwan': 'Cygnus',
    'Tundra': 'Cygnus columbianus'
}

commonname_dict = {
    'Aleutian': 'Aleutian cackling goose',
    'Cackling': 'Cackling (minima) cackling goose',
    'Dusky': 'Dusky Canada goose',
    'Less': 'Lesser Canada goose',
    'Tav': "Taverner's cackling goose",
    'TavLess': "Taverner's cackling x Lesser Canada goose",
    'Western': 'Western Canada goose',
    'Wusky': 'Western x Dusky Canada goose',
    'UnknownCackler': 'Unknown cackling goose',
    'UnknownCanada': 'Unknown Canada goose',
    'Ross': "Ross's goose",
    'Snow': 'Snow goose',
    'WhiteFronted': 'White-fronted goose',
    'UnknownGoose': 'Unknown goose',
    'Trumpeter': 'Trumpeter swan',
    'UnknownSwan': 'Unknown swan',
    'Tundra': 'Tundra swan'
}

FWSTaxonCode_dict = {
    'Aleutian': '604316',
    'Cackling': '604318',
    'Dusky': '77707',
    'Less': '76633',
    'Tav': '604320',
    'TavLess': '',
    'Western': '76632',
    'Wusky': '',
    'UnknownCackler': '604603',
    'UnknownCanada': '76625',
    'Ross': '77746',
    'Snow': '77742',
    'WhiteFronted': '77723',
    'UnknownGoose': '76624',
    'Trumpeter': '76618',
    'UnknownSwan': '76609',
    'Tundra': '76612'
}

ITIS_dict = {
    'Aleutian': '714726',
    'Cackling': '714727',
    'Dusky': '175006',
    'Less': '175004',
    'Tav': '714728',
    'TavLess': '',
    'Western': '175003',
    'Wusky': '',
    'UnknownCackler': '714068',
    'UnknownCanada': '174999',
    'Ross': '175041',
    'Snow': '175038',
    'WhiteFronted': '175020',
    'UnknownGoose': '174998',
    'Trumpeter': '174992',
    'UnknownSwan': '174984',
    'Tundra': '174987'
}

dfFlockPivot = dfFlockPivotTemp.copy()
dfFlockPivot['scientificName'] = dfFlockPivot['shortName'].map(sciname_dict)
dfFlockPivot['commonName'] = dfFlockPivot['shortName'].map(commonname_dict)
dfFlockPivot['fwsTaxonCode'] = dfFlockPivot['shortName'].map(FWSTaxonCode_dict)
dfFlockPivot['ITISTaxonCode'] = dfFlockPivot['shortName'].map(ITIS_dict)

In [25]:
### Create data frame of survey metadata for merging with pivotted long flock data
dfCleanMetadata = (
    pd.DataFrame(sedfMetadataYYYY_ObservationPoint)
    .select_columns(['globalid_x', 'globalid_y', 'SiteName', 'ObserverText', 'EffortDate_Text', 'EffortTime', 'LocationDescription', 'LocationOther', 'EffortNotes', 'LatitudeDD', 'LongitudeDD'])
)

In [26]:
### Merge survey metadata with pivotted long flock data
dfFlockLong = pd.merge(dfCleanMetadata,dfFlockPivot, how="inner", left_on="globalid_y", right_on="globalid")

In [27]:
# Reorder, clean, and rename columns
dfFlockLong = (
    pd.DataFrame(dfFlockLong)
    .select_columns(['globalid_x', 'globalid_y', 'SiteName', 'ObserverText', 'EffortDate_Text', 'EffortTime', 'LocationDescription', 'LocationOther', 'shortName', 'commonName', 'scientificName', 'fwsTaxonCode', 'ITISTaxonCode', 'value','LatitudeDD', 'LongitudeDD', 'EffortNotes'])
    .rename_columns(new_column_names={"globalid_x": "eventID", "globalid_y": "occurrenceID", "SiteName": "location", "EffortTime": "eventTime", "EffortDate_Text": "eventDate", "EffortNotes": "eventRemarks", "LatitudeDD": "decimalLatitude", "LongitudeDD": "decimalLongitude", "ObserverText": "recordedBy", "value": 'individualCount'})
)
dfFlockLong

,eventID,occurrenceID,location,recordedBy,eventDate,eventTime,LocationDescription,LocationOther,shortName,commonName,scientificName,fwsTaxonCode,ITISTaxonCode,individualCount,decimalLatitude,decimalLongitude,eventRemarks
0,c4b960e1-5770-45da-b722-c7b240377069,505c5601-051d-496a-a141-0179d1535d1c,Willapa NWR,William Ritchie,10/04/2021,12:07,Riekkola South Field #3 (Parker Sl.),None,UnknownCanada,Unknown Canada goose,Branta canadensis,76625,174999,5.0,46.365704,-124.006726,"obs from stop 1.1, not seen at stop 1.2"
1,f9390f14-6389-46a1-85bd-deff0aad6632,033ec2a0-c33b-494d-8696-0620b696ced5,Julia Butler Hansen Refuge for the Columbian W...,Paul Meyers,10/06/2021,13:00,None,None,Western,Western Canada goose,Branta canadensis moffitti,76632,175003,9.0,46.246708,-123.426036,None
2,f9390f14-6389-46a1-85bd-deff0aad6632,b4ac29dc-5c19-405d-9a80-2349153f2096,Julia Butler Hansen Refuge for the Columbian W...,Paul Meyers,10/06/2021,14:00,None,None,Western,Western Canada goose,Branta canadensis moffitti,76632,175003,110.0,46.259521,-123.439587,None
3,f9390f14-6389-46a1-85bd-deff0aad6632,b4ac29dc-5c19-405d-9a80-2349153f2096,Julia Butler Hansen Refuge for the Columbian W...,Paul Meyers,10/06/2021,14:00,None,None,Cackling,Cackling (minima) cackling goose,Branta hutchinsii minima,604318,714727,1.0,46.259521,-123.439587,None
4,f9390f14-6389-46a1-85bd-deff0aad6632,b4ac29dc-5c19-405d-9a80-2349153f2096,Julia Butler Hansen Refuge for the Columbian W...,Paul Meyers,10/06/2021,14:00,None,None,WhiteFronted,White-fronted goose,Anser albifrons,77723,175020,3.0,46.259521,-123.439587,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1873,75c1c529-a4f8-4cbb-afda-cfd50d4ed76f,8f2f1159-32eb-4b58-8cd2-b71e80fc116b,Julia Butler Hansen Refuge for the Columbian W...,Stephanie Honcharenko,4/18/2022,07:59,None,None,Western,Western Canada goose,Branta canadensis moffitti,76632,175003,2.0,46.232409,-123.402388,None
1874,75c1c529-a4f8-4cbb-afda-cfd50d4ed76f,3afaa7c2-05fd-4e15-ba7e-12de574d0a17,Julia Butler Hansen Refuge for the Columbian W...,Stephanie Honcharenko,4/18/2022,08:02,None,None,Western,Western Canada goose,Branta canadensis moffitti,76632,175003,3.0,46.241310,-123.403597,None
1875,75c1c529-a4f8-4cbb-afda-cfd50d4ed76f,106c3bd9-a471-405f-b46b-1b8fc8ec219a,Julia Butler Hansen Refuge for the Columbian W...,Stephanie Honcharenko,4/18/2022,08:21,None,None,Western,Western Canada goose,Branta canadensis moffitti,76632,175003,7.0,46.253136,-123.433117,None
1876,75c1c529-a4f8-4cbb-afda-cfd50d4ed76f,a64476e0-38ea-4291-b3b3-488b511b64cc,Julia Butler Hansen Refuge for the Columbian W...,Stephanie Honcharenko,4/18/2022,08:38,None,None,Western,Western Canada goose,Branta canadensis moffitti,76632,175003,1.0,46.234221,-123.410579,None


In [28]:
# Create wide flock data, e.g., species counts in columns not rows
dfFlockWide = (
    pd.DataFrame(sedfMetadataYYYY_ObservationPoint)
    .select_columns(['globalid_x', 'globalid_y', 'SiteName', 'ObserverText', 'EffortDate_Text', 'EffortTime', 'LocationDescription', 'LocationOther', 'DuskyCount', 'WuskyCount', 'WesternCount', 'TavLessCount', 'TavCount', 'LessCount', 'AleutianCount', 'CacklingCount', 'UnknownCanadaCount', 'UnknownCacklerCount', 'WhiteFrontedCount', 'SnowCount', 'RossCount', 'UnknownGooseCount', 'TrumpeterCount', 'TundraCount', 'UnknownSwanCount', 'Total', 'LatitudeDD', 'LongitudeDD', 'EffortNotes'])
    .rename_columns(new_column_names={"globalid_x": "eventID", "globalid_y": "occurrenceID", "SiteName": "location", "EffortTime": "eventTime", "EffortDate_Text": "eventDate", "EffortNotes": "eventRemarks", "LatitudeDD": "decimalLatitude", "LongitudeDD": "decimalLongitude", "ObserverText": "recordedBy"})
)
dfFlockWide

,eventID,occurrenceID,location,recordedBy,eventDate,eventTime,LocationDescription,LocationOther,DuskyCount,WuskyCount,...,SnowCount,RossCount,UnknownGooseCount,TrumpeterCount,TundraCount,UnknownSwanCount,Total,decimalLatitude,decimalLongitude,eventRemarks
0,c4b960e1-5770-45da-b722-c7b240377069,75497ba2-41a0-4e3d-9da8-55404c50970b,Willapa NWR,William Ritchie,10/04/2021,11:34,Riekkola Salt Marsh SE,None,0.0,0.0,...,0,0,0.0,0,0,0,0.0,46.365221,-123.994647,None
1,c4b960e1-5770-45da-b722-c7b240377069,7cc6aae9-3d1e-49d8-a617-e0fc1dfe8226,Willapa NWR,William Ritchie,10/04/2021,11:49,Riekkola Salt Marsh NE,None,0.0,0.0,...,0,0,0.0,0,0,0,0.0,46.370991,-123.995393,None
2,c4b960e1-5770-45da-b722-c7b240377069,505c5601-051d-496a-a141-0179d1535d1c,Willapa NWR,William Ritchie,10/04/2021,12:07,Riekkola South Field #3 (Parker Sl.),None,0.0,0.0,...,0,0,0.0,0,0,0,5.0,46.365704,-124.006726,"obs from stop 1.1, not seen at stop 1.2"
3,c4b960e1-5770-45da-b722-c7b240377069,ea7a4817-7cf7-419c-b11a-87a7644a117c,Willapa NWR,William Ritchie,10/04/2021,12:20,Riekkola South Field #1 (67th Place),None,0.0,0.0,...,0,0,0.0,0,0,0,0.0,46.362190,-123.995814,None
4,c4b960e1-5770-45da-b722-c7b240377069,a6d6f117-6782-4a35-8f60-5cc13e006082,Willapa NWR,William Ritchie,10/04/2021,12:49,Omeara Point (incl. Quarry Marsh),None,0.0,0.0,...,0,0,0.0,0,0,0,0.0,46.403080,-123.951040,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1037,75c1c529-a4f8-4cbb-afda-cfd50d4ed76f,8f2f1159-32eb-4b58-8cd2-b71e80fc116b,Julia Butler Hansen Refuge for the Columbian W...,Stephanie Honcharenko,4/18/2022,07:59,None,None,0.0,0.0,...,0,0,0.0,0,0,0,2.0,46.232409,-123.402388,None
1038,75c1c529-a4f8-4cbb-afda-cfd50d4ed76f,3afaa7c2-05fd-4e15-ba7e-12de574d0a17,Julia Butler Hansen Refuge for the Columbian W...,Stephanie Honcharenko,4/18/2022,08:02,None,None,0.0,0.0,...,0,0,0.0,0,0,0,3.0,46.241310,-123.403597,None
1039,75c1c529-a4f8-4cbb-afda-cfd50d4ed76f,106c3bd9-a471-405f-b46b-1b8fc8ec219a,Julia Butler Hansen Refuge for the Columbian W...,Stephanie Honcharenko,4/18/2022,08:21,None,None,0.0,0.0,...,0,0,0.0,0,0,0,7.0,46.253136,-123.433117,None
1040,75c1c529-a4f8-4cbb-afda-cfd50d4ed76f,a64476e0-38ea-4291-b3b3-488b511b64cc,Julia Butler Hansen Refuge for the Columbian W...,Stephanie Honcharenko,4/18/2022,08:38,None,None,0.0,0.0,...,0,0,0.0,0,0,0,1.0,46.234221,-123.410579,None


In [29]:
### Export csvs. Uncomment to write location-specific flock and neckband csvs

## Create separate csvs for each unique location for long flock data
#for c in dfFlockLong.location.unique():
#    (dfFlockLong[dfFlockLong.location == c]).to_csv(os.path.join(out_workspace,(c + '_Goose_Mark-Resight_FlockLong_' + timestamp + '.csv')), index= False)
    
## Create separate csvs for each unique location for wide flock data
#for c in dfFlockWide.location.unique():
#    (dfFlockWide[dfFlockWide.location == c]).to_csv(os.path.join(out_workspace,(c + '_Goose_Mark-Resight_FlockWide_' + timestamp + '.csv')), index= False)

## Create separate csvs for each unique location for dusky neckband data
#for c in dfDuskyNeckbandNWR.location.unique():
#    (dfDuskyNeckbandNWR[dfDuskyNeckbandNWR.location == c]).to_csv(os.path.join(out_workspace,(c + '_Goose_Mark-Resight_Neckband_' + timestamp + '.csv')), index= False)

## Create csv of dusky neckband data for Migratory Birds
dfDuskyNeckbandMB.to_csv(os.path.join(out_workspace,('USFWS_Goose_Mark-Resight_Neckband_' + timestamp + '.csv')), index= False)

In [30]:
### Export Excel spreadsheets

## Create separate Excel spreadsheets for each unique location
for c in sedfMetadataYYYY_ObservationPoint.SiteName.unique():
    writer = pd.ExcelWriter(os.path.join(out_workspace,(c + '_Goose_Data_' + timestamp + '.xlsx')))
    dfFlockWide[dfFlockWide.location == c].to_excel(writer, 'Flock Wide', index=False)    
    dfFlockLong[dfFlockLong.location == c].to_excel(writer, 'Flock Long', index=False)
    dfDuskyNeckbandNWR[dfDuskyNeckbandNWR.location == c].to_excel(writer, 'Dusky Neckband', index=False)
    writer.save()